In [1]:
from function import *

/home/wmbio/anaconda3/envs/deepdep/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wmbio/anaconda3/envs/deepdep/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wmbio/anaconda3/envs/deepdep/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/wmbio/anaconda3/envs/deepdep/lib/python3

In [2]:
print(os.getcwd())
os.chdir("/home/wmbio/WORK/gitworking/Dependency_prediction/prediction")
print(os.getcwd())

/home/wmbio/WORK/gitworking/Dependency_prediction/prediction/src
/home/wmbio/WORK/gitworking/Dependency_prediction/prediction


* **Path**

In [7]:
MODEL_PATH = "DepOI_1298_model/2022-0802/"
MODEL_NATM = "model_mut_exp"
PREDICT_PATH = "../preprocessing/PREDICTION/2022-08-03/"
BARCODE = "exp-mut-cna-meth" + "_" + "wmbio_ccls_"

'''
example of barcode

exp-mut-cna-meth
exp-mut-cna
exp-mut
mut-cna-meth

'''

'\nexample of barcode\n\nexp-mut-cna-meth\nexp-mut-cna\nexp-mut\nmut-cna-meth\n\n'

* **Model load**

In [4]:
with tf.device('/cpu:0'):
    model_saved = models.load_model(MODEL_PATH + "%s.h5" % MODEL_NATM)

* **Predict Set load**

In [8]:
# load TCGA genomics data and gene fingerprints
data_mut, data_labels_mut, sample_names_mut, gene_names_mut = load_data_prediction(PREDICT_PATH + BARCODE + "mut_prediction.txt")
data_exp, data_labels_exp, sample_names_exp, gene_names_exp = load_data_prediction(PREDICT_PATH + BARCODE + "exp_prediction.txt")
data_cna, data_labels_cna, sample_names_cna, gene_names_cna = load_data_prediction(PREDICT_PATH + BARCODE + "cna_prediction.txt")
data_meth, data_labels_meth, sample_names_meth, gene_names_meth = load_data_prediction(PREDICT_PATH + BARCODE + "meth_prediction.txt")
data_fprint_DepOIs, data_labels_fprint, gene_names_fprint, function_names_fprint = load_data_prediction(PREDICT_PATH + BARCODE + "fingerprint_prediction.txt")
print("\n\nDatasets successfully loaded.\n\n")



Datasets successfully loaded.




* **Prediction**

In [9]:
batch_size = 200
with tf.device('/cpu:0'):
    t = time.time()
    data_pred = np.zeros((data_exp.shape[0], data_fprint_DepOIs.shape[0]))
    for z in np.arange(0, data_exp.shape[0]):
        data_pred_tmp = model_saved.predict([
            data_mut[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_exp[np.repeat(z, data_fprint_DepOIs.shape[0])],
            # data_cna[np.repeat(z, data_fprint_DepOIs.shape[0])],
            # data_meth[np.repeat(z, data_fprint_DepOIs.shape[0])],
            data_fprint_DepOIs
        ], 
            batch_size=batch_size, verbose=0)
        data_pred[z] = np.transpose(data_pred_tmp)
        print("Unscreended sample %d predicted..." % z)

Unscreended sample 0 predicted...
Unscreended sample 1 predicted...
Unscreended sample 2 predicted...
Unscreended sample 3 predicted...
Unscreended sample 4 predicted...
Unscreended sample 5 predicted...
Unscreended sample 6 predicted...
Unscreended sample 7 predicted...
Unscreended sample 8 predicted...
Unscreended sample 9 predicted...
Unscreended sample 10 predicted...
Unscreended sample 11 predicted...
Unscreended sample 12 predicted...
Unscreended sample 13 predicted...
Unscreended sample 14 predicted...
Unscreended sample 15 predicted...
Unscreended sample 16 predicted...
Unscreended sample 17 predicted...
Unscreended sample 18 predicted...
Unscreended sample 19 predicted...
Unscreended sample 20 predicted...
Unscreended sample 21 predicted...
Unscreended sample 22 predicted...
Unscreended sample 23 predicted...
Unscreended sample 24 predicted...
Unscreended sample 25 predicted...
Unscreended sample 26 predicted...


In [10]:
# write prediction results to txt
data_pred_df = pd.DataFrame(data=np.transpose(data_pred), index=gene_names_fprint, columns=sample_names_mut[0:data_exp.shape[0]])

In [11]:
data_pred_df

,BT20,BT474,HCC2218,NCIH508,SW1417,SCC15,A498,A704,ACHN,CALU3,...,OVCAR4,CAPAN2,PC3,A101D,COLO829,MEWO,SKMEL28,SKMEL3,SKMEL5,MKN1
AARS2,-0.659024,-0.625750,-0.699582,-0.620582,-0.674384,-0.721562,-0.703286,-0.732311,-0.715633,-0.741989,...,-0.712649,-0.704933,-0.707084,-0.720373,-0.777638,-0.559472,-0.663003,-0.659030,-0.732079,-0.730950
AASDHPPT,-0.559155,-0.533300,-0.544544,-0.426701,-0.529806,-0.503974,-0.563640,-0.508426,-0.508160,-0.595919,...,-0.574493,-0.533326,-0.565525,-0.518373,-0.597123,-0.458168,-0.520534,-0.509223,-0.466684,-0.578638
ABCE1,-1.722687,-1.711432,-1.678827,-1.606356,-1.668227,-1.729081,-1.773957,-1.685390,-1.722103,-1.685226,...,-1.757892,-1.708899,-1.735916,-1.635687,-1.654060,-1.880514,-1.715565,-1.673935,-1.617687,-1.759299
ABCG1,-0.116012,-0.111884,-0.124762,-0.139625,-0.126467,-0.116120,-0.114777,-0.120539,-0.118850,-0.121928,...,-0.113080,-0.122396,-0.120238,-0.124058,-0.122907,-0.149098,-0.121347,-0.124974,-0.129232,-0.112099
ABHD11,-0.251074,-0.257603,-0.287592,-0.361221,-0.292361,-0.257108,-0.258085,-0.273627,-0.266066,-0.269084,...,-0.248185,-0.269438,-0.253975,-0.322209,-0.280436,-0.229384,-0.281042,-0.307081,-0.311021,-0.247935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF384,-0.071405,-0.069473,-0.073641,-0.075915,-0.073626,-0.069798,-0.068600,-0.073100,-0.070434,-0.074490,...,-0.069215,-0.073910,-0.071763,-0.076974,-0.075293,-0.146266,-0.074585,-0.075841,-0.076261,-0.068943
ZNF407,-0.671370,-0.642672,-0.695287,-0.779679,-0.700701,-0.679727,-0.661688,-0.688026,-0.688512,-0.688377,...,-0.651498,-0.690643,-0.678597,-0.709118,-0.695061,-0.614528,-0.679200,-0.705582,-0.660604,-0.657250
ZNF521,-0.001476,0.001734,-0.009440,-0.022774,-0.011352,-0.001155,-0.001097,-0.004546,-0.003305,-0.005356,...,0.000741,-0.005401,-0.002773,-0.011147,-0.005939,-0.013693,-0.007194,-0.011960,-0.009165,0.001285
ZNHIT2,-1.798508,-1.838919,-1.812781,-1.893133,-1.817495,-1.738666,-1.755473,-1.773113,-1.766209,-1.799398,...,-1.747825,-1.755963,-1.738567,-1.839010,-1.799647,-1.907680,-1.815530,-1.825346,-1.860463,-1.754923


In [13]:
data_pred_df.to_csv('custom_mut_exp_cna_meth.csv')